# Prendre en main l'API Python d'ArcGIS - épisode 2bis
## Planifier l'exécution du Notebook

In [1]:
from arcgis.gis import GIS
gis = GIS("home")

/opt/conda/lib/python3.9/site-packages/arcgis/gis/__init__.py:703: UserWarning: You are logged on as plouis_esrifrance with an administrator role, proceed with caution.
  warnings.warn(


In [2]:
utilisateurs = gis.users.search(max_users=1000)
#la limite d'utilisateur retournés (ici 1000, par défaut 100) est modifiable avec l'argument max_users
#Les réponses sont limitées à 10 000 résultats par requête

In [3]:
def check_user_credits_agol_noprint(gis: GIS, username: str) -> None:
    if "admin" in gis.__dict__:
        users = gis.users.search(username)
        if users:
            user = users[0]
            username_result = user.username
            if gis.admin.credits.is_enabled:
                assigned = user.assignedCredits
                available = user.availableCredits
                return(assigned,available)
            #else:
                #credits = gis.admin.credits.credits
                #print(f"This ArcGIS Online organization has disabled credit budgeting tools. The total remaining credits are: {credits}.")
        #else:
            #print(f"L'utilisateur [{username}] est introuvable.")
    #else:
        #print("Please connect to the ArcGIS Online organization as an administrator to check credits.")  

In [4]:
liste_10=[]
liste_50=[]

for utilisateur in utilisateurs :
    try :
        assigned,available = check_user_credits_agol_noprint(gis, utilisateur.username)
        if available < 0.10 * assigned:
            #print(f"Attention: les crédits disponibles pour l'utilisateur {utilisateur.username} sont inférieurs à 10% des crédits assignés ({assigned-available}/{assigned} crédits consommés - {available} crédits restants).")
            liste_10.append(utilisateur.username)
        elif available < 0.5 * assigned:
            #print(f"Les crédits disponibles pour l'utilisateur {utilisateur.username} sont inférieurs à 50% des crédits assignés ({assigned-available}/{assigned} crédits consommés - {available} crédits restants).")
            liste_50.append(utilisateur.username)
    except :
        pass

In [5]:
admin_contacts = [gis.users.me] #la notification est simplement envoyée à l'utilisateur

In [6]:
if len(liste_10)>0 or len(liste_50)>0 :
    gis.users.send_notification(admin_contacts, "Notification - utilisation des crédits de votre organisation ArcGIS Online",#titre du message
                            f"Attention: les crédits disponibles pour les utilisateurs {liste_10} sont inférieurs à 10% des crédits assignés. Notez égalempent que les crédits disponibles pour les utilisateurs {liste_50} sont inférieurs à 50% des crédits assignés.", #contenu du message
                            "email" #à retirer pour avoir une notification sur le portail
                           )